<a href="https://colab.research.google.com/github/debugginglife365/GAS_Practice/blob/main/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
listOfFrames = []

In [ ]:
# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "'1iipu2E8VIy1t47UtoYL0m5TzkvxyfOKR' in parents and mimeType='application/vnd.google-apps.spreadsheet'"}).GetList()
for file in listed:
    worksheet = gc.open_by_key(file['id']).get_worksheet(0)
    rows = worksheet.get_all_values()
    df = pd.DataFrame.from_records(rows[1:],columns=rows[0])
    listOfFrames.append(df)

print(file['id'])
print('title {}, id {}'.format(file['title'], file['id']))

1Iv2cyw8PjnMPw6eq2tH7OW5pGbWsigIKLE9rHnzfN1U
title Backup_20230607, id 1Iv2cyw8PjnMPw6eq2tH7OW5pGbWsigIKLE9rHnzfN1U


In [ ]:
combinedDF = pd.concat(listOfFrames)
combinedDF

,Account,AuxStatus,AuxReason,Start_Timestamp,End_Timestamp,Duration,Date,Week,Month,Name,Wave
0,doha.kim@tdcx.com,Start,,6/14/2023 7:49:17,6/14/2023 8:00:05,12:10:48 AM,6/14/2023,6/12/2023,6/1/2023 7:49:17,Doha Kim,
1,soojin.kim@tdcx.com,Start,,6/14/2023 7:58:03,6/14/2023 8:13:57,12:15:54 AM,6/14/2023,6/12/2023,6/1/2023 7:58:03,Soojin Kim,
2,doha.kim@tdcx.com,Helpshift,,6/14/2023 8:00:05,6/14/2023 9:00:31,1:00:26 AM,6/14/2023,6/12/2023,6/1/2023 8:00:05,Doha Kim,
3,soojin.kim@tdcx.com,Helpshift,,6/14/2023 8:13:57,6/14/2023 10:40:59,2:27:02 AM,6/14/2023,6/12/2023,6/1/2023 8:13:57,Soojin Kim,
4,Junseong.hwang@tdcx.com,Start,,6/14/2023 8:48:23,6/14/2023 8:49:11,12:00:48 AM,6/14/2023,6/12/2023,6/1/2023 8:48:23,Junseong Hwang,
...,...,...,...,...,...,...,...,...,...,...,...
128,junsu.park@tdcx.com,Helpshift,,6/7/2023 20:30:34,6/7/2023 22:00:38,1:30:04 AM,6/7/2023,6/5/2023,6/1/2023 20:30:34,Jun Su Park,
129,woongcheol.shin@tdcx.com,Break,,6/7/2023 20:34:42,6/7/2023 20:47:58,12:13:16 AM,6/7/2023,6/5/2023,6/1/2023 20:34:42,Woong Cheol Shin,
130,woongcheol.shin@tdcx.com,Helpshift,,6/7/2023 20:47:58,6/7/2023 22:00:44,1:12:46 AM,6/7/2023,6/5/2023,6/1/2023 20:47:58,Woong Cheol Shin,
131,junsu.park@tdcx.com,End,,6/7/2023 22:00:38,,,6/7/2023,6/5/2023,6/1/2023 22:00:38,Jun Su Park,


In [ ]:
df = pd.DataFrame(combinedDF)

In [ ]:
import numpy as np
df = df.replace([np.inf,-np.inf],np.nan)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = pd.to_datetime(df['Month'])

In [ ]:
df = df[df['Date'].notna()]

In [ ]:
df.drop_duplicates()
df.dropna(how='all', inplace=True)
df = df.sort_values(by='Date', ascending=False)

<ipython-input-29-ebd2c9d22706>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(how='all', inplace=True)


In [ ]:
df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')
df['Month'] = df['Month'].dt.strftime('%m/%d/%Y')

In [ ]:
wsMaster = gc.open_by_key("1N7xhmlNfGcHGHbw34wLi_SPyZoU13N0kBPh69MHCZjw").worksheet("Master")

In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe

set_with_dataframe(wsMaster, df)